# Tree search

## Code

In [87]:
import typing
import math
import collections
import dataclasses
import abc
import itertools

T = typing.TypeVar("T", int, float)
Vector = typing.Tuple[T, T]
VectorComponent = typing.List[T]
Vertex = typing.List[typing.Tuple[T, T]]

def product_combination(iterable):
    first = iterable[0]

    for r in itertools.product(first, *iterable[1:]):
        yield r


def check_axis_intersect(that: Vector, z: T):
    x, y = that
    return y >= z >= x


def is_collision(that: Vertex, other: VectorComponent) -> bool:
    if len(that) != len(other):
        raise ValueError("Invalid arguments, different length of values")

    return all([check_axis_intersect(t, z) for t, z in zip(that, other)])


@dataclasses.dataclass
class VectorNode:
    data: VectorComponent
    value: T

    def __gt__(self, other):
        return self.value > other.value

    def __str__(self):
        return f"{self.data}, {self.value}"

    def __iter__(self):
        return iter([*self.data, self.value])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        return self.vertex[item]

    def __setitem__(self, key, value):
        self.vertex[key] = value


def create_axis(_array: Vertex):
    all_data = []

    for arr in _array:
        x, y = arr
        d = math.fabs(y - x) / 2

        all_data.append(((x, x + d), (x + d, y)))

    return [*product_combination(all_data)]


@dataclasses.dataclass
class Node:
    vertex: Vertex
    nodes: list

    def append(self, node: VectorComponent):
        self.nodes.append(node)

    def is_collide(self, node: VectorComponent):
        return is_collision(self.vertex, node)

    def __len__(self):
        return len(self.nodes)

    def __iter__(self):
        return iter(self.nodes)



class NodeContainerInterface(abc.ABC):
    @property
    @abc.abstractmethod
    def axis(self) -> Vertex: ...

    @property
    @abc.abstractmethod
    def children(self): ...

    @property
    @abc.abstractmethod
    def node(self) -> Node: ...

    @property
    @abc.abstractmethod
    def is_parent(self) -> bool: ...

    @abc.abstractmethod
    def insert(self, verx: VectorComponent): ...

    @abc.abstractmethod
    def sort(self): ...

    @abc.abstractmethod
    def __iter__(self): ...


class NodeContainer:
    def __init__(self, data: Vertex, limit_divisions=1):
        self.__node = Node(vertex=data, nodes=[])
        self.__children = None
        self.__axis = data

        if limit_divisions > 0:
            self.__children = [
                NodeContainer(vertices, limit_divisions - 1) for vertices in create_axis(data)
            ]

    @property
    def axis(self):
        return [*self.__axis]

    @property
    def children(self) -> typing.List[NodeContainerInterface]:
        if self.__children:
            return [*self.__children]

        return []

    @property
    def node(self) -> Node:
        return self.__node

    @property
    def is_parent(self):
        return len(self.children) != 0

    def insert(self, verx: VectorComponent):
        return self._insert_loop(verx)

    def _insert_loop(self, verx: VectorComponent):
        children = collections.deque(self.children)

        while True:
            child = children.pop()
            node = child.node

            if node.is_collide(verx):
                if child.is_parent:
                    children = child.children

                    continue
                else:
                    node.append(verx)
                    return child

            if len(children) <= 0:
                return None

    def _insert_recursive(self, verx: VectorComponent):
        for child in self.children:
            node = child.node

            if node.is_collide(verx):
                if child.is_parent:
                    return child.insert(verx)
                else:
                    node.append(verx)
                    return child

        return None

    def sort(self):
        return self._get_iter_child_recursive()

    def __iter__(self):
        return iter(self._get_iter_child_recursive())

    def _get_iter_child_recursive(self):
        def get_iter_child(root, nodes=None):
            if nodes is None:
                nodes = []

            for child in root.children:
                node = child.node
                if child.is_parent:
                    get_iter_child(child, nodes)
                else:
                    if len(child.node) > 0:
                        nodes.append(child.node)

            return nodes

        return get_iter_child(self, [])

    def _get_iter_child_loop(self):
        pass


class TreeNode:
    def __init__(self):
        pass


## Initialize and config data

In [110]:
import sympy as sp
import numpy as np

DATA_SIZE = 10
N_COMPONENTS = 3

syms = sp.symbols(f'x1:{N_COMPONENTS +1 }')

ALL_SYMS = syms
LIMITS_RANGE = -3, 3

def get_n_syms(_range=(-1, 1)):
    return [_range for _ in range(len(syms))]

In [111]:
import operator
import functools

x1 = syms[0]
x2 = syms[1]
x3 = syms[2]

eq = sum(syms)
# eq = eq.subs(x1, 1/x1**2).subs(x2, x2**2).subs(x3, x3**3)
# eq = functools.reduce(operator.mul, syms, 1)**2 / 100

OBJ_FUNC = sp.lambdify(syms, eq, modules='numpy')

eq

x1 + x2 + x3

## Execution

In [75]:
node = NodeContainer(((0, 1), (0, 1)), 2)
node.insert((.51, .6)).node
node.insert((.1, .1)).node
node.insert((.1, .6)).node

Node(vertex=((0, 0.25), (0.5, 0.75)), nodes=[(0.1, 0.6)])

In [76]:
node.sort()

[Node(vertex=((0, 0.25), (0, 0.25)), nodes=[(0.1, 0.1)]),
 Node(vertex=((0, 0.25), (0.5, 0.75)), nodes=[(0.1, 0.6)]),
 Node(vertex=((0.5, 0.75), (0.5, 0.75)), nodes=[(0.51, 0.6)])]

In [116]:
import collections
import pandas as pd

def tree_search_algorithm(f, axis, n_it=10, n_p=5, n_limit=0):
    all_data_it = collections.deque()
    local_min_point = collections.deque()
    tree = NodeContainer(axis, 2)

    for it in range(n_it):
        for _ in range(n_p):
            data = [np.random.uniform(*_) for _ in tree.axis]
            value = f(*data)

            tree.insert(VectorNode(data, value))

        for container in tree.sort():
            nodes = container.nodes
            print(container)

        all_data_it.append(
            VectorNode(data, value)
        )

    return [list(vec) for vec in all_data_it], min(all_data_it)

all_data, min_found = tree_search_algorithm(
    OBJ_FUNC, [(-3, 3) for _ in range(len(syms))], 1, 20
)

min_found

Node(vertex=((-3, -1.5), (-3, -1.5), (-1.5, 0.0)), nodes=[VectorNode(data=[-1.7042849794964998, -2.446215756532184, -0.11824313654857921], value=-4.2687438725772635)])
Node(vertex=((-3, -1.5), (-1.5, 0.0), (-3, -1.5)), nodes=[VectorNode(data=[-2.4432643805138143, -1.2260702772373178, -2.7649212394848313], value=-6.434255897235963)])
Node(vertex=((-3, -1.5), (-1.5, 0.0), (-1.5, 0.0)), nodes=[VectorNode(data=[-2.7966779257063075, -0.8773917656740231, -0.9300726460822784], value=-4.6041423374626085)])
Node(vertex=((-1.5, 0.0), (-3, -1.5), (0.0, 1.5)), nodes=[VectorNode(data=[-1.0605039110245007, -2.0577298853961254, 0.034831721469589905], value=-3.0834020749510365)])
Node(vertex=((-1.5, 0.0), (-3, -1.5), (1.5, 3)), nodes=[VectorNode(data=[-0.13719414386235274, -1.6917613401776708, 2.1988519826286357], value=0.3698964985886122)])
Node(vertex=((-1.5, 0.0), (-1.5, 0.0), (1.5, 3)), nodes=[VectorNode(data=[-0.4789672475016271, -0.856369390792477, 2.739886000576904], value=1.4045493622828)])
No

VectorNode(data=[1.2541560242290029, 2.012024841843563, 0.11425705842675438], value=3.3804379244993203)

In [61]:
pd.DataFrame(all_data)

,0,1,2,3
0,-1.066253,-0.120718,-0.523206,0.000023
1,-2.541996,0.038567,2.505100,0.000302
2,-0.446239,-0.997476,2.345128,0.005448
3,-0.696158,0.671550,-2.619270,0.007497
4,1.572645,-2.575518,-0.829626,0.056458
5,-0.015733,1.236107,-2.961319,0.000017
6,-2.570403,0.172742,-1.037122,0.001060
7,1.793059,0.878113,-2.844859,0.100319
8,-0.720859,0.264582,1.069696,0.000208
9,2.698649,-0.941268,-1.485869,0.071228


In [35]:
min_found

VectorNode(data=[-2.6208587062665307, -2.933830523368222, -2.9205574323905594], value=-8.475246662025313)